# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## PART II. Exceution of Queries on Apache Cassandra

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifyapp 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the newly created keyspace
session.set_keyspace('sparkifyapp')

## Create queries to ask the following three questions of the data


#### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)

```For the first query Primary Key, sessionId is used as partion key and itemInSession is used as clustering key. This allows Data partitioning by sessionId and clustering by itemInSession which helps in fast retrieval of data for a specific sessionId.```

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songs (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

##### Test Sql Query for Query 1

In [10]:
testQuery= "select artist, song, length from songs where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(testQuery)
except Exception as e:
    print(e)

try:
    for row in rows:
        print (row.artist, '|',row.song,'|', row.length)
except Exception as e:
    print(e)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


#### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
query = "CREATE TABLE IF NOT EXISTS songplay_songdata "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)

```For the second query Primary Key, userid is used as partition key and sessionId is used as clustering key. Due to this data is partitioned by userId and clustered by sessionId. This helps in fast retrieval of data for specific userId. And data is clustered by sessionId so it returns data sorted by sessionId```

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplay_songdata (userId, sessionId, itemInSession, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

##### Test Sql Query for Query 2

In [13]:
testQuery= "select artist, song, first_name, last_name from songplay_songdata where userId=10 and sessionId=182"
try:
    rows = session.execute(testQuery)
except Exception as e:
    print(e)

try:
    for row in rows:
        print (row.artist, '|',row.song, '|', row.first_name, ' ', row.last_name)
except Exception as e:
    print(e)


Down To The Bone | Keep On Keepin' On | Sylvie   Cruz
Three Drives | Greece 2000 | Sylvie   Cruz
Sebastien Tellier | Kilometer | Sylvie   Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie   Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
query = "CREATE TABLE IF NOT EXISTS songplay_userdata "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
session.execute(query)

```For the third query Primary Key, song name is selected as partion key which allows easy retrieval of data for specific song name, but this only can't identify data uniquely in table. So user_id is used as clustering key, as user id is unique for every user. This makes each row unique and all data for specific song name can easily fetched from database.``` 

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplay_userdata (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

##### Test Sql Query for Query 3

In [16]:
testQuery= "select first_name, last_name from songplay_userdata where song='All Hands Against His Own'"
try:
    rows = session.execute(testQuery)
except Exception as e:
    print(e)

try:
    for row in rows:
        print (row.first_name,' ' ,row.last_name)
except Exception as e:
    print(e)

Jacqueline   Lynch
Tegan   Levine
Sara   Johnson


### Drop the tables before closing out the sessions

In [17]:
drop_songs = """DROP TABLE IF EXISTS songs"""
drop_songplay_songdata = """DROP TABLE IF EXISTS songplay_songdata"""
drop_songplay_userdata = """DROP TABLE IF EXISTS songplay_userdata"""

In [18]:
try:
    session.execute(drop_songs)
    session.execute(drop_songplay_songdata)
    session.execute(drop_songplay_userdata)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()